In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [3]:
df_orig = pd.read_csv("train.csv")
print(df_orig.shape)
print("Null Counts:")
print(df_orig.isnull().sum()[df_orig.isnull().sum() > 0])

(1460, 81)
Null Counts:
LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [4]:
df = df_orig.drop(columns=["MiscFeature", "Fence", "PoolQC", "Alley", "Id", "SalePrice"])
print(df.shape)

(1460, 75)


In [5]:
def fill_nulls(df, mean, mode):
    df.fillna(mean, inplace=True)
    df.fillna(mode, inplace=True)

mean = df.mean()
mode = df.mode().iloc[0]
fill_nulls(df, mean, mode)
print(df.isnull().sum()[df.isnull().sum() > 0])

Series([], dtype: int64)


In [6]:
def encode_categorical_features(df, enc):
    # Flatten enc.categories_ which is a list of np.array
    cat_list = np.concatenate(enc.categories_).ravel()
    df[cat_list] = pd.DataFrame(enc.transform(df[categorical_features]).toarray(), index=df.index)
    df = df.drop(columns=categorical_features)
    return df

categorical_features = df.select_dtypes(exclude=[np.number]).columns
onehot = OneHotEncoder(handle_unknown='ignore')
onehot.fit(df[categorical_features])
df = encode_categorical_features(df, onehot)
print(df.shape)

(1460, 203)


In [7]:
def standardize_numerical_features(df):
    numerical_features = df.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    return df

df = standardize_numerical_features(df)

In [42]:
X = df
y = df_orig.SalePrice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1314, 203) (1314,)
(146, 203) (146,)


In [51]:
reg = 0.1
kreg = 10000.0
model = Sequential()
model.add(Dense(units=90, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(kreg)))
#model.add(Dropout(reg))
#model.add(Dense(units=90, activation='relu'))
# model.add(Dropout(reg))
#model.add(Dense(units=90, activation='relu'))
# model.add(Dropout(reg))
# model.add(Dense(units=90, activation='relu'))
# model.add(Dropout(reg))
# model.add(Dense(units=90, activation='relu'))
# model.add(Dropout(reg))
# model.add(Dense(units=90, activation='relu'))
# model.add(Dropout(reg))
# model.add(Dense(units=90, activation='relu'))
# model.add(Dropout(reg))
model.add(Dense(units=1, activation=None, kernel_regularizer=l2(kreg)))

model.compile(loss='mean_squared_error',
              optimizer=Adam(0.1),
              metrics=['mean_squared_logarithmic_error'])

model.fit(X_train, y_train, epochs=200, batch_size=16)

Epoch 1/100
1314/1314 [==============================] - 7s 5ms/step - loss: 11002636197.4064 - mean_squared_logarithmic_error: 7.2647
Epoch 2/100
1314/1314 [==============================] - 3s 2ms/step - loss: 3396832596.5540 - mean_squared_logarithmic_error: 0.2103
Epoch 3/100
1314/1314 [==============================] - 3s 2ms/step - loss: 2301179785.9361 - mean_squared_logarithmic_error: 0.0485
Epoch 4/100
1314/1314 [==============================] - 3s 2ms/step - loss: 2458933639.5982 - mean_squared_logarithmic_error: 0.0453
Epoch 5/100
1314/1314 [==============================] - 3s 2ms/step - loss: 2138186729.9848 - mean_squared_logarithmic_error: 0.0456
Epoch 6/100
1314/1314 [==============================] - 3s 3ms/step - loss: 1913936643.7017 - mean_squared_logarithmic_error: 0.0358
Epoch 7/100
1314/1314 [==============================] - 3s 3ms/step - loss: 2416912276.2618 - mean_squared_logarithmic_error: 0.0490A: 0s - loss: 2569263902.3729 - mean_squared_logarithm
Epoch 8

1314/1314 [==============================] - 3s 2ms/step - loss: 940129523.0441 - mean_squared_logarithmic_error: 0.0182
Epoch 55/100
1314/1314 [==============================] - 3s 2ms/step - loss: 845977141.9665 - mean_squared_logarithmic_error: 0.0152
Epoch 56/100
1314/1314 [==============================] - 3s 2ms/step - loss: 1013118383.7321 - mean_squared_logarithmic_error: 0.0193A: 1s - loss: 1034946542.6667 - mean_squared_logari
Epoch 57/100
1314/1314 [==============================] - 3s 3ms/step - loss: 905683254.1613 - mean_squared_logarithmic_error: 0.0154
Epoch 58/100
1314/1314 [==============================] - 3s 3ms/step - loss: 830567906.2892 - mean_squared_logarithmic_error: 0.0154
Epoch 59/100
1314/1314 [==============================] - ETA: 0s - loss: 843508103.1111 - mean_squared_logarithmic_error: 0.01 - 3s 3ms/step - loss: 842823902.8798 - mean_squared_logarithmic_error: 0.0158
Epoch 60/100
1314/1314 [==============================] - 3s 2ms/step - loss: 9171998

In [52]:
train_loss, train_msle = model.evaluate(X_train, y_train)
test_loss, test_msle = model.evaluate(X_test, y_test)
print(np.sqrt(train_loss), np.sqrt(test_loss))
print(np.sqrt(train_msle), np.sqrt(test_msle))
# Current best is 0.14

# No reg
# 5153.652146570697 37264.89209303301
# 0.02738456283225992 0.2730824166735976

# 0.1 Dropout
# 9237.206300457883 35590.03618316048
# 0.04418546037015819 0.2115214128827394

# 0.1 Dropout .2 test split
# 9167.790059515022 33455.80715732241
# 0.04495856600453306 0.1992055513045245

# # 0.1 Dropout .1 test split
# 10031.654561233698 26025.607890733154
# 0.0490407746056158 0.20793296973375155

# 0.1 DO .1 TS random_state=42
# 8697.668314355573 31722.346238090824
# 0.04483347051854697 0.1762138780849231

# 10000 kreg
# 36049.5586146419 42306.33210159457
# 0.1220954378651295 0.1320486500045756

146/146 [==============================] - 0s 739us/step
25384.630289379096 34648.679427680814
0.10197781580761467 0.14853294426706393


In [47]:
df_test_orig = pd.read_csv("test.csv")
df_test = df_test_orig.drop(columns=["MiscFeature", "Fence", "PoolQC", "Alley", "Id"])
fill_nulls(df_test, mean, mode)
df_test = encode_categorical_features(df_test, onehot)
df_test = standardize_numerical_features(df_test)
predictions = model.predict(df_test)
predictions = np.squeeze(predictions)
print(predictions.shape)
df_submit = pd.DataFrame({'Id': df_test_orig.Id, 'SalePrice': predictions})
df_submit.to_csv('submission.csv', index=False)

(1459,)
